# RAG Application using Type Sense

In [4]:
import typesense
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
client = typesense.Client({
    'nodes': [{
        'host': os.getenv("TYPESENSE_HOST"),
        'port':'443', # Typesense default value is 443
        'protocol': 'https' # Typesense default - https
    }],
    'api_key': os.getenv("TYPESENSE_API_KEY"),
    'connection_timeout_seconds': 2
})

In [6]:
client

In [7]:
# Defining Books schema (books.json)
books_schema = {
    'name': 'books',
    'fields': [
        {'name': 'title', 'type':'string'},
        {'name': 'authors', 'type':'string[]', 'facet':True},
        {'name': 'publication_year', 'type':'int32', 'facet': True},
        {'name': 'ratings_count', 'type':'int32'},
        {'name': 'average_rating', 'type':'float'},
    ],
    'default_sorting_field':'ratings_count'
}

print(client.collections.create(books_schema))

{'created_at': 1767283488, 'default_sorting_field': 'ratings_count', 'enable_nested_fields': False, 'fields': [{'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'title', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'authors', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string[]'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'publication_year', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'ratings_count', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'average_rating', 'optional': False, 'sort': True, 'stem': Fal

In [ ]:
with open('books.jsonl', 'r', encoding='utf-8') as jsonl_file:
    data = jsonl_file.read()
    client.collections['books'].documents.import_(data)

In [11]:
# creating search parameters
search_parameters = {
    'q' : 'harry potter',
    'query_by': 'title,authors',
    'sort_by':'ratings_count:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 17,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}},
  {'document': {'authors': ['J.K. Rowling', ' Mary GrandPré', ' R